In [11]:
import csv as csv
import numpy as np

In [12]:
csv_file_object = csv.reader(open("train.csv",'rb'))
header = csv_file_object.next()
data = []

In [13]:
for row in csv_file_object:
    data.append(row)
data = np.array(data)

In [46]:
fare_ceiling = 40

data[ data[0::,9].astype(np.float) >= fare_ceiling, 9] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size
number_of_classes = 3
number_of_classes = len(np.unique(data[0::,2]))

In [47]:
survival_table = np.zeros([2,number_of_classes,number_of_price_brackets],float)

In [48]:
for i in xrange(number_of_classes):
    for j in xrange(number_of_price_brackets):

        women_only_stats = data[ (data[0::,4] == "female") \
                                 & (data[0::,2].astype(np.float) == i+1) \
                                 & (data[0:,9].astype(np.float) >= j*fare_bracket_size) \
                                 & (data[0:,9].astype(np.float) < (j+1)*fare_bracket_size), 1]

        men_only_stats = data[ (data[0::,4] != "female") \
                                 & (data[0::,2].astype(np.float) == i+1) \
                                 & (data[0:,9].astype(np.float) >= j*fare_bracket_size) \
                                 & (data[0:,9].astype(np.float) < (j+1)*fare_bracket_size), 1]

                                 #if i == 0 and j == 3:

        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float))  # Female stats
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))    # Male stats


In [49]:
survival_table[ survival_table != survival_table ] = 0.

In [50]:
survival_table

array([[[ 0.        ,  0.        ,  0.83333333,  0.97727273],
        [ 0.        ,  0.91428571,  0.9       ,  1.        ],
        [ 0.59375   ,  0.58139535,  0.33333333,  0.125     ]],

       [[ 0.        ,  0.        ,  0.4       ,  0.38372093],
        [ 0.        ,  0.15873016,  0.16      ,  0.21428571],
        [ 0.11153846,  0.23684211,  0.125     ,  0.24      ]]])

In [51]:
survival_table[survival_table < 0.5] = 0
survival_table[survival_table >= 0.5] = 1

In [52]:
survival_table

array([[[ 0.,  0.,  1.,  1.],
        [ 0.,  1.,  1.,  1.],
        [ 1.,  1.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]])

In [64]:
test_file = open("test.csv", "rb")
test_file_object = csv.reader(test_file)
header = test_file_object.next()
predictions_file = open("genderclassmodel.csv", "wb")
p = csv.writer(predictions_file)
p.writerow(["PassengerId","Survived"])

In [65]:
for row in test_file_object: # Iterates through each row in the the test file object
    
    # finds a bin_fare
    for j in xrange(number_of_price_brackets): # 0,1,2,3
        
        try:
            
            row[8] = float(row[8]) # attempts to make the fare a float
        except:
            bin_fare = 3 - float(row[1]) # if it fails it sets bin_fare = 3-Pclass i.e. 3-Pclass(3rd) = 0
            break
        if row[8] > fare_ceiling:
            
            bin_fare = number_of_price_brackets - 1 # if fare greater than 40 then bin_fare = 4-1 = 3
            break
        
        # if fare greater than (0,1,2,3)*10 and less than (1,2,3,4)*10, bin_fare = (0,1,2,3)
        if row[8] >= j * fare_bracket_size and row[8] < (j+1) * fare_bracket_size: # i.e.fare=15 => 1*10 and (1+1)*10
            bin_fare = j                                                           #          15 is between (10-20) so 
            break                                                                  #          bin_fare = j = 1


    if row[3] == 'female':
        # write to file the Passenger Id row[0], and the value in the survival_table
        # if fare = 15, bin would be 1. float of Pclass - 1 ie. 2
        # so survival_table[0,2,1]
        p.writerow([row[0], '%d' % \
                    int(survival_table[0, float(row[1])-1, bin_fare])])
    else:
        p.writerow([row[0], "%d" % \
                  int(survival_table[1, float(row[1])-1, bin_fare])])

test_file.close()
predictions_file.close()
                
        

C:\Users\Director UYE\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Director UYE\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1.0